In [5]:
import sys
sys.path.append('../../src')
from energiapy.components.result import Result
import pandas
from itertools import product
from matplotlib import rc
import matplotlib.pyplot as plt
from energiapy.model.solve import solve
from energiapy.plot.plot_results import CostY, CostX
from energiapy.plot import plot_results, plot_scenario, plot_location
from energiapy.utils.nsrdb_utils import fetch_nsrdb_data
from energiapy.model.formulate import formulate, Constraints, Objective
from energiapy.model.bounds import CapacityBounds
from energiapy.utils.data_utils import get_data, make_henry_price_df, remove_outliers, load_results, calculate_hourly
from energiapy.components.transport import Transport
from energiapy.components.scenario import Scenario
from energiapy.components.network import Network
from energiapy.components.location import Location
from energiapy.components.material import Material
from energiapy.components.case_study import CaseStudy
from energiapy.components.process import Process, VaryingProcess
from energiapy.components.resource import Resource, VaryingResource
from energiapy.components.temporal_scale import TemporalScale
import numpy
from energiapy.conversion.photovoltaic import solar_power_output
from energiapy.conversion.windmill import wind_power_output
from energiapy.aggregation.reduce_scenario import reduce_scenario, Clustermethod
from energiapy.aggregation.ahc import agg_hierarchial_elbow, IncludeAHC, Fit

In [2]:
horizon = 1

In [8]:
#San Diego data

# fetch_nsrdb_data(attrs=['dni', 'dhi', 'wind_speed', 'ghi', 'air_temperature', 'dew_point', 'relative_humidity', 'surface_pressure'], year=2016 , state='California',
#                                     county='San Diego', resolution='hourly', get='min-elevation')
# weather_sandiego = pandas.concat([fetch_nsrdb_data(attrs=['dni', 'dhi', 'wind_speed', 'ghi', 'air_temperature', 'dew_point', 'relative_humidity', 'surface_pressure'], year=2016+i , state='California',
#                                     county='San Diego', resolution='hourly', get='min-elevation')[1] for i in range(5)])
# index = weather_sandiego.index
# weather_sandiego.index = pandas.to_datetime(weather_sandiego.index, utc = True)
# weather_sandiego.index = weather_sandiego.index.strftime('%m/%d/%Y, %r')
# weather_sandiego = weather_sandiego[~weather_sandiego.index.str.contains('02/29')]

In [11]:
coord_houston = (29.56999969482422, -95.05999755859375)
weather_houston = pandas.read_csv('./data/weather_houston.csv', index_col=0)
weather_houston.index = pandas.to_datetime(weather_houston.index, utc = True)

C:\Users\shivam.vedant\AppData\Local\Temp\ipykernel_18548\2622049418.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  weather_houston.index = pandas.to_datetime(weather_houston.index, utc = True)


In [12]:
solar_houston = solar_power_output(data= weather_houston, coord=coord_houston, sam = 'cecmod', module_params= 'Canadian_Solar_Inc__CS5P_220M', 
                                   inverter = 'cecinverter', inverter_params= 'ABB__MICRO_0_25_I_OUTD_US_208__208V_', 
                                   temperature_params= 'open_rack_glass_glass', aoi_model= 'no_loss', ac_model= 'sandia', spectral_model= 'no_loss')

In [13]:
wind_houston = wind_power_output(data= weather_houston, roughness_length= 0.1, turbine_type= 'V100/1800', hub_height= 92, 
                                 wind_speed_model= 'logarithmic', density_model = 'ideal_gas', temperature_model = 'linear_gradient', 
                                 power_output_model = 'power_coefficient_curve', density_correction = True, obstacle_height = 0, 
                                 observation_height = 10)

C:\Users\shivam.vedant\PycharmProjects\energiapy\venv_energiapy\lib\site-packages\windpowerlib\wind_turbine.py:173: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  self.nominal_power = float(turbine_data["nominal_power"])
C:\Users\shivam.vedant\PycharmProjects\energiapy\venv_energiapy\lib\site-packages\windpowerlib\wind_turbine.py:175: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  self.rotor_diameter = float(turbine_data["rotor_diameter"])


In [15]:
ng_price_df = pandas.concat(
    [make_henry_price_df(file_name='./data/Henry_Hub_Natural_Gas_Spot_Price_Daily.csv', year=2016 +i, stretch= True) for i in range(5)])

In [16]:
ercot = pandas.read_csv('./data/ercot.csv', index_col=0)
ercot.index = pandas.to_datetime(ercot.index, utc = True)
ercot.loc['2016-11-06 23:00:00'] = ercot.loc['2016-11-06 22:00:00'] #random missing value

C:\Users\shivam.vedant\AppData\Local\Temp\ipykernel_18548\2984035665.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  ercot.index = pandas.to_datetime(ercot.index, utc = True)


In [18]:
url = 'https://oedi-data-lake.s3.amazonaws.com/ATB/electricity/parquet/2022/ATBe.parquet'
raw_data = pandas.read_parquet(url)
raw_data = raw_data.astype(
    dtype={
        'core_metric_key': 'string',
        'core_metric_parameter': 'string',
        'core_metric_case': 'string',
        'crpyears': 'string',
        'technology': 'string',
        'technology_alias': 'string',
        'techdetail': 'string',
        'display_name': 'string',
        'scenario': 'string',
        'units': 'string'
    })

In [19]:
raw_data['technology'].unique()

<StringArray>
[                'NaturalGas_FE',    'Commercial Battery Storage',
     'Pumped Storage Hydropower',   'Residential Battery Storage',
 'Utility-Scale Battery Storage',                    'Hydropower',
                      'Biopower',                           'CSP',
                        'CommPV',               'DistributedWind',
                    'Geothermal',                 'LandbasedWind',
                  'OffShoreWind',                         'ResPV',
                     'UtilityPV', 'Utility-Scale PV-Plus-Battery',
                       'Nuclear',                       'Coal_FE',
                           'AEO']
Length: 19, dtype: string

In [20]:
def atb_gttr(core_metric_parameters, core_metric_case, crpyear, technology, techdetail, scenario):
    df_out = pandas.DataFrame()
    for i in core_metric_parameters:
        df = pandas.DataFrame(raw_data[
            (raw_data.core_metric_parameter == i) &
            (raw_data.core_metric_case == core_metric_case) &
            (raw_data.crpyears == str(crpyear)) &
            (raw_data.technology == technology) &
            (raw_data.techdetail == techdetail) &
            (raw_data.scenario == scenario)
        ][['value']])
        df = df.rename({'value': i}, axis='columns')
        df = df.reset_index()
        df_out = pandas.concat([df_out, df], axis='columns')
        df_out = df_out.loc[:, ~df_out.columns.duplicated()].copy()
        df_out = df_out.fillna(0)
        df_out = df_out.drop(columns=['index'])
    return df_out


In [21]:

hig_trl_adv = [(1 - i/(31*15)) for i in range(31)]
hig_trl_mod = [(1 - i/(31*10)) for i in range(31)]
hig_trl_con = [(1 - i/(31*5)) for i in range(31)]

med_trl_adv = [(1 - i/(31*40)) for i in range(31)]
med_trl_mod = [(1 - i/(31*30)) for i in range(31)]
med_trl_con = [(1 - i/(31*20)) for i in range(31)]

low_trl_adv = [(1 - i/(31*70)) for i in range(31)]
low_trl_mod = [(1 - i/(31*50)) for i in range(31)]
low_trl_con = [(1 - i/(31*30)) for i in range(31)]

hig_trl_adv_df = pandas.DataFrame(
    data={'CAPEX': hig_trl_adv, 'Fixed O&M': hig_trl_adv, 'Variable O&M': hig_trl_adv})
hig_trl_mod_df = pandas.DataFrame(
    data={'CAPEX': hig_trl_mod, 'Fixed O&M': hig_trl_mod, 'Variable O&M': hig_trl_mod})
hig_trl_con_df = pandas.DataFrame(
    data={'CAPEX': hig_trl_con, 'Fixed O&M': hig_trl_con, 'Variable O&M': hig_trl_con})
med_trl_adv_df = pandas.DataFrame(
    data={'CAPEX': med_trl_adv, 'Fixed O&M': med_trl_adv, 'Variable O&M': med_trl_adv})
med_trl_mod_df = pandas.DataFrame(
    data={'CAPEX': med_trl_mod, 'Fixed O&M': med_trl_mod, 'Variable O&M': med_trl_mod})
med_trl_con_df = pandas.DataFrame(
    data={'CAPEX': med_trl_con, 'Fixed O&M': med_trl_con, 'Variable O&M': med_trl_con})
low_trl_adv_df = pandas.DataFrame(
    data={'CAPEX': low_trl_adv, 'Fixed O&M': low_trl_adv, 'Variable O&M': low_trl_adv})
low_trl_mod_df = pandas.DataFrame(
    data={'CAPEX': low_trl_mod, 'Fixed O&M': low_trl_mod, 'Variable O&M': low_trl_mod})
low_trl_con_df = pandas.DataFrame(
    data={'CAPEX': low_trl_con, 'Fixed O&M': low_trl_con, 'Variable O&M': low_trl_con})
constant_df = pandas.DataFrame(
    data={'CAPEX': [1]*31, 'Fixed O&M': [1]*31, 'Variable O&M': [1]*31})


param_list = ['CAPEX', 'Fixed O&M', 'Variable O&M']
advanced_dict = {
    'RPV': atb_gttr(core_metric_parameters=param_list, core_metric_case='Market', crpyear=30, technology='ResPV', techdetail='Class5', scenario='Advanced'),
    'PV': atb_gttr(core_metric_parameters=param_list, core_metric_case='Market', crpyear=30, technology='UtilityPV', techdetail='Class1', scenario='Advanced'),
    'WF': atb_gttr(core_metric_parameters=param_list, core_metric_case='Market', crpyear=30, technology='LandbasedWind', techdetail='Class1', scenario='Advanced'),
    'LiI': atb_gttr(core_metric_parameters=param_list, core_metric_case='R&D', crpyear=30, technology='Commercial Battery Storage', techdetail='8Hr Battery Storage', scenario='Advanced'),
    'LII_discharge': constant_df,
    'PSH': atb_gttr(core_metric_parameters=param_list, core_metric_case='Market', crpyear=30, technology='Pumped Storage Hydropower', techdetail='NatlClass10', scenario='Advanced'),
    'PSH_discharge': constant_df,
    'ASMR': atb_gttr(core_metric_parameters=param_list, core_metric_case='Market', crpyear=30, technology='Nuclear', techdetail='NuclearSMR', scenario='Moderate'),
    'NPP': atb_gttr(core_metric_parameters=param_list, core_metric_case='Market', crpyear=30, technology='Nuclear', techdetail='Nuclear', scenario='Advanced'),
    'NGCC': atb_gttr(core_metric_parameters=param_list, core_metric_case='Market', crpyear=30, technology='NaturalGas_FE', techdetail='CCCCSHFrame95%', scenario='Conservative'),
    'SMR': hig_trl_adv_df,
    'SMRH': hig_trl_adv_df,
    'AqOff': hig_trl_adv_df,
    'DAC': low_trl_adv_df,
    'PEM': low_trl_adv_df,
    'H2_Comp': med_trl_adv_df,
}


moderate_dict = {
    'PV': atb_gttr(core_metric_parameters=param_list, core_metric_case='Market', crpyear=30, technology='UtilityPV', techdetail='Class1', scenario='Moderate'),
    'WF': atb_gttr(core_metric_parameters=param_list, core_metric_case='Market', crpyear=30, technology='LandbasedWind', techdetail='Class1', scenario='Moderate'),
    'LiI': atb_gttr(core_metric_parameters=param_list, core_metric_case='R&D', crpyear=30, technology='Commercial Battery Storage', techdetail='8Hr Battery Storage', scenario='Moderate'),
    'LII_discharge': constant_df,
    'PSH': atb_gttr(core_metric_parameters=param_list, core_metric_case='Market', crpyear=30, technology='Pumped Storage Hydropower', techdetail='NatlClass10', scenario='Moderate'),
    'PSH_discharge': constant_df,
    'ASMR': atb_gttr(core_metric_parameters=param_list, core_metric_case='Market', crpyear=30, technology='Nuclear', techdetail='NuclearSMR', scenario='Moderate'),
    'NPP': atb_gttr(core_metric_parameters=param_list, core_metric_case='Market', crpyear=30, technology='Nuclear', techdetail='Nuclear', scenario='Moderate'),
    'NGCC': atb_gttr(core_metric_parameters=param_list, core_metric_case='Market', crpyear=30, technology='NaturalGas_FE', techdetail='CCCCSHFrame95%', scenario='Conservative'),
    'SMR': hig_trl_mod_df,
    'SMRH': hig_trl_mod_df,
    'AqOff': hig_trl_mod_df,
    'DAC': low_trl_mod_df,
    'PEM': low_trl_mod_df,
    'H2_Comp': med_trl_mod_df,
}


conservative_dict = {
    'PV': atb_gttr(core_metric_parameters=param_list, core_metric_case='Market', crpyear=30, technology='UtilityPV', techdetail='Class1', scenario='Conservative'),
    'WF': atb_gttr(core_metric_parameters=param_list, core_metric_case='Market', crpyear=30, technology='LandbasedWind', techdetail='Class1', scenario='Conservative'),
    'LiI': atb_gttr(core_metric_parameters=param_list, core_metric_case='R&D', crpyear=30, technology='Commercial Battery Storage', techdetail='8Hr Battery Storage', scenario='Conservative'),
    'LII_discharge': constant_df,
    'PSH': atb_gttr(core_metric_parameters=param_list, core_metric_case='Market', crpyear=30, technology='Pumped Storage Hydropower', techdetail='NatlClass10', scenario='Conservative'),
    'PSH_discharge': constant_df,
    # no cons, adv
    'ASMR': atb_gttr(core_metric_parameters=param_list, core_metric_case='Market', crpyear=30, technology='Nuclear', techdetail='NuclearSMR', scenario='Moderate'),
    'NPP': atb_gttr(core_metric_parameters=param_list, core_metric_case='Market', crpyear=30, technology='Nuclear', techdetail='Nuclear', scenario='Conservative'),
    'NGCC': atb_gttr(core_metric_parameters=param_list, core_metric_case='Market', crpyear=30, technology='NaturalGas_FE', techdetail='CCCCSHFrame95%', scenario='Conservative'),
    'SMR': hig_trl_con_df,
    'SMRH': hig_trl_con_df,
    'AqOff': hig_trl_con_df,
    'DAC': low_trl_con_df,
    'PEM': low_trl_con_df,
    'H2_Comp': med_trl_con_df,
}


In [22]:
scales = TemporalScale(discretization_list=[horizon, 365, 24])

In [23]:
bigM = 10**5
smallM = 10

In [24]:
credit = {'DAC': {0: 31.77, 1: 31.77, 2: 31.77, 3: 31.77, 4: 31.77, 5: 50.00, 6: 50.00, 7: 50.00, 8: 50.00, 9: 50.00},
          'AqOff': {0: 20.22, 1: 20.22, 2: 20.22, 3: 20.22, 4: 20.22, 5: 35.00, 6: 35.00, 7: 35.00, 8: 35.00, 9: 35.00}
}

In [25]:
def model(name: str, year:int, cost_dict:dict, credit: dict = None, pathway: str = None):
    
    if pathway is None:
        pathway = 'full'
    
    #*---------------- Resources---------------------------------

    Solar = Resource(name='Solar', cons_max=100, basis='MW', label='Solar Power')

    Wind = Resource(name='Wind', cons_max=100, basis='MW', label='Wind Power')

    Power = Resource(name='Power', basis='MW', label='Power generated')#, demand = True)

    Uranium = Resource(name='Uranium', cons_max=100,
                    price=42.70/(250/2), basis='kg', label='Uranium')

    H2 = Resource(name='H2', basis='tons', label='Hydrogen')

    H2_C = Resource(name='H2(C)', basis='tons', label='Hydrogen(C)', demand=True, revenue= 0.005)

    CO2_AQoff = Resource(name='CO2_AQoff', basis='tons',
                        label='Carbon dioxide - sequestered', sell=True)

    H2O = Resource(name='H2O', cons_max=1000,
                price=3.00, basis='tons', label='Water', block='Resource')

    CH4 = Resource(name='CH4', cons_max=1000, price=max(ng_price_df['CH4'])*1000, basis='tons',
                label='Natural gas', varying=[VaryingResource.DETERMINISTIC_PRICE])

    CO2 = Resource(name='CO2', basis='tons',
                label='Carbon dioxide', block='Resource')

    CO2_Vent = Resource(name='CO2_Vent', sell=True, basis='tons',
                        label='Carbon dioxide - Vented')

    O2 = Resource(name='O2', sell=True, basis='tons', label='Oxygen')

    CO2_DAC = Resource(name='CO2_DAC', basis='tons',
                    label='Carbon dioxide - captured')

    LiI_Power = Resource(name='LiI_Power', basis='MW',
                        label='Power in LiI', store_max=10000, store_min=smallM)

    PSH_Power = Resource(name='PSH_Power', basis='MW',
                        label='Power in PSH', store_max=10000, store_min=smallM)

    #*---------------- Processes---------------------------------


    PEM = Process(name='PEM', conversion={Power: -1, H2: 0.3537, O2: 2.8302, H2O: -3.1839}, capex= cost_dict['PEM']['CAPEX'][year]*1.55*10**6, 
                 fopex=0, vopex=0, prod_max=100, prod_min = 0, basis = 'tons', label = 'PEM' )

    # LiI = Process(name='LiI', storage=Power, capex=1302182, fopex=41432, vopex=2000,
    #             prod_max=100, prod_min=0, store_max=10000, label='Lithium-ion battery', basis='MW')


    LiI = Process(name='LiI', conversion={Power: -1, LiI_Power: 1}, capex= cost_dict['LiI']['CAPEX'][year]*1000, 
                 fopex=cost_dict['LiI']['Fixed O&M'][year]*1000, vopex=cost_dict['LiI']['Variable O&M'][year]*1000,
                store_max=10000, store_min=smallM, prod_max=bigM, prod_min=smallM, label='Lithium-ion battery', basis='MW', block='energy_storage')

    LiI_discharge = Process(name='LiI', conversion={Power: 1, LiI_Power: -1}, capex=0.1,
                            fopex=0.01, vopex=0, prod_max=bigM, prod_min=smallM,  label='Lithium-ion battery (d)', basis='MW')

    WF = Process(name='WF', conversion={Wind: -1, Power: 1}, capex= cost_dict['WF']['CAPEX'][year]*1000, 
                 fopex=cost_dict['WF']['Fixed O&M'][year]*1000, vopex=cost_dict['WF']['Variable O&M'][year]*1000, land = 10800/1800,
                prod_max=100, prod_min=0, label='Wind mill array', varying=[VaryingProcess.DETERMINISTIC_CAPACITY], basis='MW')

    PV = Process(name='PV', conversion={Solar: -1, Power: 1}, capex= cost_dict['PV']['CAPEX'][year]*1000, 
                 fopex=cost_dict['PV']['Fixed O&M'][year]*1000, vopex=cost_dict['PV']['Variable O&M'][year]*1000,  land = 10000/1800,
                prod_max=100, prod_min=0,  varying= [VaryingProcess.DETERMINISTIC_CAPACITY], label='Solar PV', basis='MW')

    H2_Comp = Process(name='H2_Comp', conversion={Power: -0.417, H2_C: 1, H2: -1}, capex= cost_dict['H2_Comp']['CAPEX'][year]*1.6 *
                 10**6, fopex= 0, vopex= 0, prod_max=100, prod_min=0,  label='Hydrogen Compression')

    SMRH = Process(name='SMRH', conversion={Power: -1.11, CH4: -3.76, H2O: -23.7, H2: 1, CO2_Vent: 1.03, CO2: 9.332}, capex= cost_dict['SMRH']['CAPEX'][year]*252000, 
                 fopex=cost_dict['SMRH']['Fixed O&M'][year]*945000, vopex=cost_dict['SMRH']['Variable O&M'][year]*51.5,
                prod_max=bigM, prod_min=smallM,  label='Steam methane reforming + CCUS', block='h2_prod')

    NGCC = Process(name='NGCC', conversion={Power: 1, CH4: -0.108, CO2_Vent: 0.297*0.05, CO2: 0.297 *
                0.95},capex= cost_dict['NGCC']['CAPEX'][year]*1000, 
                 fopex=cost_dict['NGCC']['Fixed O&M'][year]*1000, vopex=cost_dict['NGCC']['Variable O&M'][year]*1000, prod_max=bigM, prod_min=smallM,  label='NGCC + 95% CC', block='power_gen')


    NG = Process(name='NG', conversion={Power: 1, CH4: -0.108, CO2_Vent: 0.297},capex= 0.3*cost_dict['NGCC']['CAPEX'][year]*1000, 
                 fopex= 0.3*cost_dict['NGCC']['Fixed O&M'][year]*1000, vopex= 0.3*cost_dict['NGCC']['Variable O&M'][year]*1000, prod_max=bigM, prod_min=smallM,  label='NGCC + 95% CC', block='power_gen')


    SMR = Process(name='SMR', capex= cost_dict['SMR']['CAPEX'][year]*240000, 
                 fopex=cost_dict['SMR']['Fixed O&M'][year]*800000, vopex=cost_dict['SMR']['Variable O&M'][year]*30,  conversion={
                Power: -1.11, CH4: -3.76, H2O: -23.7, H2: 1, CO2_Vent: 9.4979}, prod_max=bigM, prod_min=0.01,  label='Steam methane reforming', block='h2_prod')


    PSH = Process(name='PSH', conversion={Power: -1, PSH_Power: 1}, capex= cost_dict['PSH']['CAPEX'][year]*1000, 
                 fopex=cost_dict['PSH']['Fixed O&M'][year]*1000, vopex=cost_dict['PSH']['Variable O&M'][year]*1000, prod_max=bigM, prod_min=smallM, store_min=smallM, store_max=1000, label='Pumped storage hydropower', basis='MW', block='energy_storage')

    PSH_discharge = Process(name='PSH_discharge', conversion={Power: 1, PSH_Power: -1},  capex=0.1,
                            fopex=0.01, vopex=0, prod_max=bigM, prod_min=smallM,  label='Pumped storage hydropower (d)', basis='MW')

    ASMR = Process(name='ASMR', conversion={Uranium: -4.17*10**(-5), H2O: -3.364, Power: 1}, capex= cost_dict['ASMR']['CAPEX'][year]*1000, 
                 fopex=cost_dict['ASMR']['Fixed O&M'][year]*1000, vopex=cost_dict['ASMR']['Variable O&M'][year]*1000, prod_max=bigM, prod_min=smallM, label='Small modular reactors (SMRs)', block='power_gen')

    AqOff = Process(name='AqOff', conversion={Power: -1.28, CO2_AQoff: 1, CO2: -1}, capex= cost_dict['AqOff']['CAPEX'][year]*552000, 
                fopex=0, vopex=cost_dict['AqOff']['Variable O&M'][year]*4.140, prod_max=bigM, prod_min=smallM,
                label='Offshore aquifer CO2 sequestration (SMR)', block='ccus')
    
    DAC = Process(name='DAC', capex= cost_dict['DAC']['CAPEX'][year]*1000, 
                fopex=cost_dict['DAC']['Fixed O&M'][year]*1000, vopex=cost_dict['DAC']['Variable O&M'][year]*1000, conversion={
                Power: -0.193, H2O: -4.048, CO2_DAC: 1}, prod_max=bigM, prod_min=smallM,  label='Direct air capture', block='ccus')

    
    if pathway == 'full':
        
        process_set = {LiI, WF, ASMR, PEM, H2_Comp, NGCC, AqOff, PSH, SMR, SMRH, PV}
    
    if pathway == 'green':
        
        process_set = {LiI, WF, ASMR, PEM, H2_Comp, PSH, PV}
    
    if pathway == 'blue':
         
        process_set = {LiI, WF, H2_Comp, NGCC, AqOff, PSH, SMRH, PV}
    
    if pathway == 'grey':
        
        process_set = {LiI, H2_Comp, NG, PSH, SMR}
    
    #*---------------- Location---------------------------------
    
    if credit is not None:
        houston= Location(name='HO', processes=process_set, capacity_factor={PV: solar_houston[:8760*horizon], WF: wind_houston[:8760*horizon]}, price_factor={
                   CH4: ng_price_df[:8760*horizon]}, scales=scales, label='Houston (conservative)', expenditure_scale_level=0, demand_scale_level=2, capacity_scale_level=2, 
                          price_scale_level=2, land_cost = 10000, credit = {AqOff: credit['AqOff'][year], DAC: credit['DAC'][year]})
    else:
        houston= Location(name='HO', processes=process_set, capacity_factor={PV: solar_houston[:8760*horizon], WF: wind_houston[:8760*horizon]}, price_factor={
                   CH4: ng_price_df[:8760*horizon]}, scales=scales, label='Houston (conservative)', expenditure_scale_level=0, demand_scale_level=2, capacity_scale_level=2, price_scale_level=2, land_cost = 10000)

    #*---------------- Scenario---------------------------------
    
    scenario = Scenario(name=name, network=houston, scales=scales,  expenditure_scale_level=0, scheduling_scale_level=2,
                    network_scale_level=0, demand_scale_level=1, purchase_scale_level=2, capacity_scale_level = 2, label='full_case', demand={houston: {H2_C: 0.2740}}, annualization_factor = 0.1)

    scenario_reduced, repday_reduced, info_reduced =  reduce_scenario(scenario=scenario, periods=40, scale_level=1, method=Clustermethod.AHC, include = [IncludeAHC.CAPACITY, IncludeAHC.PRICE])

    
    return scenario_reduced

